<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Sistema de obtención de información con NLTK utilizando un corpus del supermercado La Gallega


In [113]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np
import pandas as pd

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image, display

[nltk_data] Downloading package punkt to /Users/jmbeitia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jmbeitia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jmbeitia/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Datos
Se consumirán los datos de todos los productos del supermercado La Gallega.

In [114]:
dfProductos = pd.read_json('datosLaGallega.json', orient='records')
dfProductos

,descripcion,precio,linkFoto
0,rocio vegetal mazola x 120 ml. manteca,363.35,https://www.lagallega.com.ar/Archivos/Articulo...
1,rocio vegetal natura x 120 ml.,363.64,https://www.lagallega.com.ar/Archivos/Articulo...
2,rocio vegetal mazola x 120 ml. original,376.65,https://www.lagallega.com.ar/Archivos/Articulo...
3,rocio vegetal de girasol lira x 120 gr.,382.70,https://www.lagallega.com.ar/Archivos/Articulo...
4,rocio vegetal fritolim x 120 ml. clasico,394.34,https://www.lagallega.com.ar/Archivos/Articulo...
...,...,...,...
4978,talco para pies rexona efficient original x 20...,472.35,https://www.lagallega.com.ar/Archivos/Articulo...
4979,polvo pedico veritas x 180 gr.,496.62,https://www.lagallega.com.ar/Archivos/Articulo...
4980,toallitas antibacteriales bialcohol x 15 unid.,212.78,https://www.lagallega.com.ar/Archivos/Articulo...
4981,pilas rayovac alcalina c x 2 unid.,138.89,https://www.lagallega.com.ar/Archivos/Articulo...


### 2 - Dividir el texto en sentencias y en palabras

In [115]:
corpus = [descripcion for descripcion in dfProductos['descripcion']]
# words = [nltk.word_tokenize(descripcion) for descripcion in corpus]
words = nltk.word_tokenize(' '.join(corpus))

In [116]:
len(corpus)

4983

In [117]:
# Demos un vistazo
corpus[:10]

['rocio vegetal mazola x 120 ml. manteca',
 'rocio vegetal natura x 120 ml.',
 'rocio vegetal mazola x 120 ml. original',
 'rocio vegetal de girasol lira x 120 gr.',
 'rocio vegetal fritolim x 120 ml. clasico',
 'rocio vegetal natura x 120 ml. oliva',
 'aceite de soja sojola x 900 ml.',
 'aceite de soja sojola x1.5lt.',
 'aceite de girasol legitimo x 1,5 lt.',
 'aceite de girasol lira x 900 ml.']

In [118]:
# Demos un vistazo
words[:20]

['rocio',
 'vegetal',
 'mazola',
 'x',
 '120',
 'ml',
 '.',
 'manteca',
 'rocio',
 'vegetal',
 'natura',
 'x',
 '120',
 'ml',
 '.',
 'rocio',
 'vegetal',
 'mazola',
 'x',
 '120']

In [119]:
print("Términos:", len(words))

Términos: 41128


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [120]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

# ord() nos da el código Unicode para un caracter dado
punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    # 1 - reduce el texto a mínuscula (string.lower())
    # 2 - quitar los simbolos de puntuacion (string.translate())
    # 3 - realiza la tokenización (nltk.word_tokenize)
    # 4 - realiza la lematización (nuestra función perform_lemmatization)
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    # word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='spanish')
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text)

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response, idx = "Lo siento, no puedo encontrar el producto buscado", None
    else:
        response, idx = corpus[similar_sentence_number], similar_sentence_number # obtener el documento del corpus más similar
    
    # corpus.remove(user_input)
    return response, idx

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias a ensayar:
- Grand slam
- tournaments
- nadal
- artificial intelligence

In [122]:
def bot_response(human_text):
    print("Busqueda:", human_text)
    resp, index_label = generate_response(human_text.lower(), list(dfProductos['descripcion']))
    if index_label:
        print("-"*10, "Resultado encontrado", "-"*50)
        print("Descripción:", resp.capitalize())
        print("Precio:", dfProductos.at[index_label, 'precio'])
        display(Image(url=dfProductos.at[index_label, 'linkFoto']))
    else:
        print("A:", resp)

In [123]:
bot_response('pepe larralde')

Busqueda: pepe larralde
A: Lo siento, no puedo encontrar el producto buscado


In [124]:
bot_response('manteca')

Busqueda: manteca
---------- Resultado encontrado --------------------------------------------------
Descripción: Manteca milkaut x 100 gr.
Precio: 223.01


In [125]:
bot_response('serenisima manteca 200')

Busqueda: serenisima manteca 200
---------- Resultado encontrado --------------------------------------------------
Descripción: Manteca la serenisima x 200 gr.
Precio: 496.78


In [126]:
bot_response('fernet 750')

Busqueda: fernet 750
---------- Resultado encontrado --------------------------------------------------
Descripción: Aperitivo fernet branca x 750 ml.
Precio: 1700.0


In [127]:
bot_response('coca cola 1.5')

Busqueda: coca cola 1.5
---------- Resultado encontrado --------------------------------------------------
Descripción: Bebida sin alcohol coca cola x 1.5 lt.
Precio: 276.0


In [128]:
bot_response('azucar chango')

Busqueda: azucar chango
---------- Resultado encontrado --------------------------------------------------
Descripción: Azucar chango en sobres x 100 unid.
Precio: 491.8


In [129]:
bot_response('oreo mini')

Busqueda: oreo mini
---------- Resultado encontrado --------------------------------------------------
Descripción: Galletitas mini oreo x 50 gr.
Precio: 101.74


In [130]:
bot_response('yogur bebible')

Busqueda: yogur bebible
---------- Resultado encontrado --------------------------------------------------
Descripción: Yogur bebible milkaut x 900 gr. vainilla
Precio: 265.48
